In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)

In [2]:
#db = sqlite3.connect('data/hop_team.sqlite')
#db.execute('CREATE INDEX from_npi ON calls(from_npi)')
#db.close()

In [3]:
query = '''WITH tax_nucc AS(
            SELECT * 
            FROM taxonomy as t
            JOIN nucc as n
            ON t.taxonomy_code=n.code
            WHERE taxonomy_switch = 'Y' 
            ),
            trim_npi AS(
            SELECT npi, entity_type_code, `provider_organization_name_(legal_business_name)`,`provider_last_name_(legal_name)`,provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_other_organization_name,provider_first_line_business_mailing_address,provider_second_line_business_mailing_address,provider_business_mailing_address_city_name,provider_business_mailing_address_state_name,provider_business_mailing_address_postal_code
            FROM npi
            WHERE provider_business_mailing_address_state_name = 'TN'
            ),
            best_npi AS(
                        SELECT*
                        FROM trim_npi
                        JOIN tax_nucc
                        on trim_npi.npi= tax_nucc.NPI
                        )
            SELECT *
            FROM calls
            JOIN best_npi
            on calls.from_npi=best_npi.npi
            JOIN best_npi AS b1
            on calls.to_npi=b1.npi
            WHERE transaction_count>49
            AND average_day_wait<100
            AND average_day_wait>0
            '''
with sqlite3.connect('data/hop_team.sqlite') as db:
    full = pd.read_sql(query, db)
    
    

In [16]:
full

,from_npi,patient_count,transaction_count,average_day_wait,std_day_wait,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_other_organization_name,provider_first_line_business_mailing_address,provider_second_line_business_mailing_address,provider_business_mailing_address_city_name,provider_business_mailing_address_state_name,provider_business_mailing_address_postal_code,NPI:1,taxonomy_switch,taxonomy_code,code,grouping,classification,specialization,definition,notes,display_name,section,to_npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_other_organization_name,provider_first_line_business_mailing_address,provider_second_line_business_mailing_address,provider_business_mailing_address_city_name,provider_business_mailing_address_state_name,provider_business_mailing_address_postal_code,NPI:1,taxonomy_switch,taxonomy_code,code,grouping,classification,specialization,definition,notes,display_name,section
0,1841293891,176,231,28.857,60.036,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,1841293891,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,Individual,1245290857,2.0,RADIOLOGY IMAGING ASSOCIATES OF OAK RIDGE PC,None,None,None,None,None,None,RADIOLOGY IMAGING ASSOCIATES OF OAK RIDGE,601 DODDS AVE,None,CHATTANOOGA,TN,374043911.0,1245290857,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,Individual
1,1841293891,44,50,35.260,60.604,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,1841293891,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,Individual,1245437060,1.0,None,MONAZAM TAVALLAEE,MEHRAN,None,None,None,M.D.,None,49 CLEVELAND ST,SUITE 350,CROSSVILLE,TN,385559716.0,1245437060,Y,207R00000X,207R00000X,Allopathic & Osteopathic Physicians,Internal Medicine,None,"A physician who provides long-term, comprehens...","Source: American Board of Medical Specialties,...",Internal Medicine Physician,Individual
2,1841293891,67,76,53.184,83.149,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,1841293891,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,Individual,1306061601,1.0,None,LEMOND,THOMAS,CHARLES,DR.,None,M.D.,None,990 OAK RIDGE TPKE,None,OAK RIDGE,TN,378306976.0,1306061601,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Radiology,"A radiologist who utilizes x-ray, radionuclide...","Source: American Board of Medical Specialties,...",Diagnostic Radiology Physician,Individual
3,1841293891,56,66,61.894,81.764,1.0,None,GIBBS,ELMER,RICKEY,DR.,None,M.D.,None,49 CLEVELAND ST 310,None,CROSSVILLE,TN,385552854.0,1841293891,Y,208600000X,208600000X,Allopathic & Osteopathic Physicians,Surgery,None,A general surgeon has expertise related to the...,"Source: American Board of Medical Specialties,...",Surgery Physician,Individual,1356633564,1.0,None,LUTTRELL,LINDSAY,GEORGE,DR.,None,M.D.,None,2508 WILD FERN LN,None,KNOXVILLE,TN,379313359.0,1356633564,Y,2085R0202X,2085R0202X,Allopathic & Osteopathic Physicians,Radiology,Diagnostic Rad

In [17]:
full.shape

(852201, 56)

In [6]:
full.info

<bound method DataFrame.info of           from_npi      to_npi  patient_count  transaction_count  \
0       1841293891  1245290857            176                231   
1       1841293891  1245437060             44                 50   
2       1841293891  1306061601             67                 76   
3       1841293891  1356633564             56                 66   
4       1841293891  1457302010            233                366   
...            ...         ...            ...                ...   
852196  1275001851  1184610859             62                 62   
852197  1275001851  1275531956            130                130   
852198  1275001851  1366440570             74                 74   
852199  1275001851  1710919980            108                108   
852200  1275001851  1972648319             84                 84   

        average_day_wait  std_day_wait         npi  entity_type_code  \
0                 28.857        60.036  1841293891               1.0   
1      

In [7]:
full.columns

Index(['from_npi', 'to_npi', 'patient_count', 'transaction_count',
       'average_day_wait', 'std_day_wait', 'npi', 'entity_type_code',
       'provider_organization_name_(legal_business_name)',
       'provider_last_name_(legal_name)', 'provider_first_name',
       'provider_middle_name', 'provider_name_prefix_text',
       'provider_name_suffix_text', 'provider_credential_text',
       'provider_other_organization_name',
       'provider_first_line_business_mailing_address',
       'provider_second_line_business_mailing_address',
       'provider_business_mailing_address_city_name',
       'provider_business_mailing_address_state_name',
       'provider_business_mailing_address_postal_code', 'NPI:1',
       'taxonomy_switch', 'taxonomy_code', 'code', 'grouping',
       'classification', 'specialization', 'definition', 'notes',
       'display_name', 'section', 'npi', 'entity_type_code',
       'provider_organization_name_(legal_business_name)',
       'provider_last_name_(legal_name

In [12]:
last_col= full.pop('to_npi')

In [13]:
full.insert(loc=31, column= 'to_npi', value= last_col)

In [15]:
full.pop('npi')

,npi,npi
0,1841293891,1245290857
1,1841293891,1245437060
2,1841293891,1306061601
3,1841293891,1356633564
4,1841293891,1457302010
...,...,...
852196,1275001851,1184610859
852197,1275001851,1275531956
852198,1275001851,1366440570
852199,1275001851,1710919980
